# Import Library

In [1]:
import pandas as pd
import json

# Load Data

In [2]:
info = pd.read_csv('Data/superhero_info - superhero_info.csv')
info.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [3]:
powers = pd.read_csv('Data/superhero_powers - superhero_powers.csv')
powers.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


# Inspect Data before Cleaning

In [4]:
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [5]:
powers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


# Clean Data

# Part 1

I. Clean the files and combine them into one final DataFrame.

This dataframe should have the following columns:
- Hero (Just the name of the Hero)
- Publisher
- Gender
- Eye color
- Race
- Hair color
- Height (numeric)
- Skin color
- Alignment
- Weight (numeric)
- Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
    - Agility
    - Flight
    - Superspeed
    - etc.
    
Hint: There is a space in "100 kg" or "52.5 cm"

## 1. 'Hero' column should just contain the name of the Hero

In [6]:
# Check the current values under the Hero|Publisher column
info['Hero|Publisher'].value_counts()

Spider-Man|Marvel Comics       3
Black Canary|DC Comics         2
Nova|Marvel Comics             2
Toxin|Marvel Comics            2
Batman|DC Comics               2
                              ..
Fabian Cortez|Marvel Comics    1
Exodus|Marvel Comics           1
Evilhawk|Marvel Comics         1
Evil Deadpool|Marvel Comics    1
Zoom|DC Comics                 1
Name: Hero|Publisher, Length: 457, dtype: int64

In [7]:
## Separate the 'Hero|Publisher' column into 2 columns and save the 2 new columns into the dataframe
info[['Hero','Publisher']] = info['Hero|Publisher'].str.split('|',expand=True)
info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [8]:
## drop the original column
info = info.drop(columns=['Hero|Publisher'])
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


## 2. Separate the "Measurements" column into "Height" (Numerical) and "Weight" (Numerical)

### 2.1  Convert a string column of dictionaries into actual dictionaries.

In [9]:
## use .str.replace to replace all single quotes
info['Measurements'] = info['Measurements'].str.replace("'",'"')
## Apply the json.loads to the full column
info['Measurements'] = info['Measurements'].apply(json.loads)
info['Measurements'].head(2)

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
Name: Measurements, dtype: object

### 2.2 Unpack a column of dictionaries into separate columns

In [10]:
measurements = info['Measurements'].apply(pd.Series)
measurements

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


### 2.3 Concatenate these 2 new columns to original dataframe and drop the original column.

In [11]:
# concat measurements with original dataframe
info = pd.concat([info,measurements], axis=1)
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [12]:
# drop the original column
info = info.drop(columns=['Measurements'])

In [13]:
# check to confirm that the measurements column has dropped
info.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg


### 2.4 Replace multiple characters at once within a string column

203.0 cm -> 203.0


Remove ' cm' from the Height column

In [14]:
# Make a list of all characters to replace
to_replace = [" cm"]
# run a loop to replace all of the characters in the list at once
for char in to_replace:
    info['Height'] = info['Height'].str.replace(char,"")
    
info['Height'].head()

0    203.0
1    191.0
2    185.0
3    203.0
4    193.0
Name: Height, dtype: object

Remove ' kg' from the Weight column

In [15]:
# Make a list of all characters to replace
to_replace = [" kg"]
# run a loop to replace all of the characters in the list at once
for char in to_replace:
    info['Weight'] = info['Weight'].str.replace(char, "")
    
info['Weight'].head()

0    441.0
1     65.0
2     90.0
3    441.0
4    122.0
Name: Weight, dtype: object

### 2.5 Convert dtype of values in "Height" and "Weight" columns to numerical

In [16]:
info['Height'] = info['Height'].astype(float)

In [17]:
info['Weight'] = info['Weight'].astype(float)

In [18]:
#check
info.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0


In [19]:
#check dtype
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Gender      463 non-null    object 
 1   Race        463 non-null    object 
 2   Alignment   463 non-null    object 
 3   Hair color  463 non-null    object 
 4   Eye color   463 non-null    object 
 5   Skin color  463 non-null    object 
 6   Hero        463 non-null    object 
 7   Publisher   463 non-null    object 
 8   Height      463 non-null    float64
 9   Weight      463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


Alternative Way: 

In [20]:
#info['Height'] = pd.to_numeric(info['Height'])

In [21]:
#info['Weight'] = pd.to_numeric(info['Weight'])

In [22]:
# check
#info.head()

In [23]:
#check dtype
#info.info()

## 3. Combine the two dataset into one dataframe

Check for common columns in two dataframes

In [24]:
info['Hero'].value_counts()

Spider-Man        3
Black Canary      2
Nova              2
Captain Marvel    2
Toxin             2
                 ..
Feral             1
Falcon            1
Fabian Cortez     1
Exodus            1
Zoom              1
Name: Hero, Length: 455, dtype: int64

In [25]:
powers['hero_names'].value_counts()

3-D Man            1
Namora             1
Naruto Uzumaki     1
Nathan Petrelli    1
Nebula             1
                  ..
Evilhawk           1
Exodus             1
Fabian Cortez      1
Falcon             1
Zoom               1
Name: hero_names, Length: 667, dtype: int64

In [26]:
#rename the 'hero_names' column name in the 'powers' dataframe to 'Hero' to match the column name
#in the 'info' dataframe

powers = powers.rename(columns={'hero_names': 'Hero'})

In [27]:
powers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Hero    667 non-null    object
 1   Powers  667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


In [28]:
hero = pd.merge(info,powers, on='Hero')
hero

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,Powers
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,"Accelerated Healing,Durability,Longevity,Super..."
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,"Agility,Accelerated Healing,Cold Resistance,Du..."
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,Lantern Power Ring
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,"Accelerated Healing,Intelligence,Super Strengt..."
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,"Cold Resistance,Durability,Energy Absorption,S..."
...,...,...,...,...,...,...,...,...,...,...,...
458,Male,Human,good,Blond,blue,Unknown,Yellowjacket,Marvel Comics,183.0,83.0,"Size Changing,Animal Oriented Powers"
459,Female,Human,good,Strawberry Blond,blue,Unknown,Yellowjacket II,Marvel Comics,165.0,52.0,"Flight,Energy Blasts,Size Changing"
460,Male,Yoda's species,good,White,brown,green,Yoda,George Lucas,66.0,17.0,"Agility,Stealth,Danger Sense,Marksmanship,Weap..."
461,Female,Human,good,Black,blue,Unknown,Zatanna,DC Comics,170.0,57.0,"Cryokinesis,Telepathy,Magic,Fire Control,Proba..."


## 4. Create a new column for each of  unique values that are in Powers column,  and One-hot-encode columns for every power that appears in the dataset.

In [29]:
hero['Powers'].value_counts()

Durability,Super Strength                                                                                                                                                                                                                                                                       4
Agility,Stealth,Marksmanship,Weapons Master,Stamina                                                                                                                                                                                                                                             4
Intelligence                                                                                                                                                                                                                                                                                    4
Agility,Accelerated Healing,Durability,Stealth,Danger Sense,Marksmanship,Animal Attributes,Super Strength,Stamina,Super Speed,Anim

In [30]:
# Split the values into separate rows
hero['Powers'] = hero['Powers'].str.split(',')
hero = hero.explode('Powers')

In [31]:
hero

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,Powers
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,Accelerated Healing
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,Durability
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,Longevity
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,Super Strength
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,Stamina
...,...,...,...,...,...,...,...,...,...,...,...
461,Female,Human,good,Black,blue,Unknown,Zatanna,DC Comics,170.0,57.0,Weather Control
462,Male,Unknown,bad,Brown,red,Unknown,Zoom,DC Comics,185.0,81.0,Super Speed
462,Male,Unknown,bad,Brown,red,Unknown,Zoom,DC Comics,185.0,81.0,Intangibility
462,Male,Unknown,bad,Brown,red,Unknown,Zoom,DC Comics,185.0,81.0,Time Travel


In [32]:
## saving the unique values from the 'Powers' column
cols_to_make = hero['Powers'].unique()

In [33]:
cols_to_make

array(['Accelerated Healing', 'Durability', 'Longevity', 'Super Strength',
       'Stamina', 'Camouflage', 'Self-Sustenance', 'Agility',
       'Cold Resistance', 'Underwater breathing', 'Marksmanship',
       'Weapons Master', 'Intelligence', 'Telepathy', 'Immortality',
       'Reflexes', 'Enhanced Sight', 'Sub-Mariner', 'Lantern Power Ring',
       'Super Speed', 'Invulnerability', 'Animation', 'Super Breath',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Molecular Manipulation',
       'Heat Resistance', 'Matter Absorption', 'Stealth', 'Flight',
       'Power Suit', 'Energy Blasts', 'Energy Beams', 'Power Cosmic',
       'Heat Generation', 'Danger Sense', 'Teleportation', 'Phasing',
       'Force Fields', 'Hypnokinesis', 'Energy Manipulation',
       'Invisibility', 'Enhanced Senses', 'Jump', 'Substance Secretion',
       'Natural Weapons', 'Wallcrawling', 'Vision - Thermal',
       'Power Augmentation', 'Cryokinesis', 'Dupli

In [34]:
## Filter ALL warnings
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# create a new column for each of our unique values that are stored in cols_to_make
# use .astype(int) to one-hot-encode (instead of displaying as Boolean)
for col in cols_to_make:
    hero[col] = hero['Powers'].str.contains(col).astype(int)
hero.head(20)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,Powers,Accelerated Healing,Durability,Longevity,Super Strength,Stamina,Camouflage,Self-Sustenance,Agility,Cold Resistance,Underwater breathing,Marksmanship,Weapons Master,Intelligence,Telepathy,Immortality,Reflexes,Enhanced Sight,Sub-Mariner,Lantern Power Ring,Super Speed,Invulnerability,Animation,Super Breath,Energy Absorption,Elemental Transmogrification,Fire Resistance,Natural Armor,Molecular Manipulation,Heat Resistance,Matter Absorption,Stealth,Flight,Power Suit,Energy Blasts,Energy Beams,Power Cosmic,Heat Generation,Danger Sense,Teleportation,Phasing,Force Fields,Hypnokinesis,Energy Manipulation,Invisibility,Enhanced Senses,Jump,Substance Secretion,Natural Weapons,Wallcrawling,Vision - Thermal,Power Augmentation,Cryokinesis,Duplication,Size Changing,Weapon-based Powers,Shapeshifting,Fire Control,Enhanced Memory,Energy Constructs,Power Absorption,Enhanced Hearing,Time Travel,Enhanced Smell,Water Control,Vision - Telescopic,Magnetism,Vision - Microscopic,Vision - Night,Vision - Heat,Vision - X-Ray,Magic,Psionic Powers,Gliding,Toxin and Disease Resistance,Animal Attributes,Animal Oriented Powers,Regeneration,Toxin and Disease Control,Empathy,Animal Control,Elasticity,Web Creation,Symbiote Costume,Telekinesis,Levitation,Biokinesis,Sonar,Enhanced Touch,Seismic Power,Energy Armor,Radiation Immunity,Density Control,Light Control,Illumination,Molecular Combustion,Dimensional Travel,Audio Control,Sonic Scream,Peak Human Condition,Vision - Infrared,Dexterity,Energy Resistance,Vision - Cryo,Clairvoyance,Probability Manipulation,Electrokinesis,Darkforce Manipulation,Illusions,Death Touch,Gravity Control,Precognition,Time Manipulation,Omnilingualism,Astral Projection,Technopath/Cyberpath,Mind Control,Mind Blast,Cloaking,Telepathy Resistance,Mind Control Resistance,Radiation Control,Radiation Absorption,Magic Resistance,Dimensional Awareness,Intangibility,Element Control,Wind Control,Echolocation,Radar Sense,Resurrection,The Force,Possession,Astral Travel,Summoning,Banish,Adaptation,Electrical Transport,Speed Force,Omnipotent,Grim Reaping,Reality Warping,Portal Creation,Vitakinesis,Projection,Insanity,Plant Control,Photographic Reflexes,Anti-Gravity,Power Nullifier,Hair Manipulation,Weather Control,Nova Force,Odin Force,Phoenix Force,Power Sense,Qwardian Power Ring,Melting,Changing Armor,Terrakinesis
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,Accelerated Healing,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,Durability,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,Longevity,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,Super Strength,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [37]:
# Drop the original column: "Powers"

hero.drop(columns=['Powers'],inplace=True)

# Final Check of the Dataframe

In [38]:
hero.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4466 entries, 0 to 462
Columns: 169 entries, Gender to Terrakinesis
dtypes: float64(2), int64(159), object(8)
memory usage: 5.9+ MB


In [42]:
# Display the first row of first 15 columns only
hero.iloc[0, :15]

Gender                          Male
Race                           Human
Alignment                       good
Hair color                   No Hair
Eye color                     yellow
Skin color                   Unknown
Hero                          A-Bomb
Publisher              Marvel Comics
Height                         203.0
Weight                         441.0
Accelerated Healing                1
Durability                         0
Longevity                          0
Super Strength                     0
Stamina                            0
Name: 0, dtype: object

**Verified that the dataframe has been cleaned**:

I. Clean the files and combine them into one final DataFrame.

The final dataframe has the following columns:
- [x] Hero (Just the name of the Hero)
- [x] Publisher
- [x] Gender
- [x] Eye color
- [x] Race
- [x] Hair color
- [x] Height (numeric)
- [x] Skin color
- [x] Alignment
- [x] Weight (numeric)
- Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
   - Agility
   - Flight
   - Superspeed
   - etc.

In [43]:
hero.head(5)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,Accelerated Healing,Durability,Longevity,Super Strength,Stamina,Camouflage,Self-Sustenance,Agility,Cold Resistance,Underwater breathing,Marksmanship,Weapons Master,Intelligence,Telepathy,Immortality,Reflexes,Enhanced Sight,Sub-Mariner,Lantern Power Ring,Super Speed,Invulnerability,Animation,Super Breath,Energy Absorption,Elemental Transmogrification,Fire Resistance,Natural Armor,Molecular Manipulation,Heat Resistance,Matter Absorption,Stealth,Flight,Power Suit,Energy Blasts,Energy Beams,Power Cosmic,Heat Generation,Danger Sense,Teleportation,Phasing,Force Fields,Hypnokinesis,Energy Manipulation,Invisibility,Enhanced Senses,Jump,Substance Secretion,Natural Weapons,Wallcrawling,Vision - Thermal,Power Augmentation,Cryokinesis,Duplication,Size Changing,Weapon-based Powers,Shapeshifting,Fire Control,Enhanced Memory,Energy Constructs,Power Absorption,Enhanced Hearing,Time Travel,Enhanced Smell,Water Control,Vision - Telescopic,Magnetism,Vision - Microscopic,Vision - Night,Vision - Heat,Vision - X-Ray,Magic,Psionic Powers,Gliding,Toxin and Disease Resistance,Animal Attributes,Animal Oriented Powers,Regeneration,Toxin and Disease Control,Empathy,Animal Control,Elasticity,Web Creation,Symbiote Costume,Telekinesis,Levitation,Biokinesis,Sonar,Enhanced Touch,Seismic Power,Energy Armor,Radiation Immunity,Density Control,Light Control,Illumination,Molecular Combustion,Dimensional Travel,Audio Control,Sonic Scream,Peak Human Condition,Vision - Infrared,Dexterity,Energy Resistance,Vision - Cryo,Clairvoyance,Probability Manipulation,Electrokinesis,Darkforce Manipulation,Illusions,Death Touch,Gravity Control,Precognition,Time Manipulation,Omnilingualism,Astral Projection,Technopath/Cyberpath,Mind Control,Mind Blast,Cloaking,Telepathy Resistance,Mind Control Resistance,Radiation Control,Radiation Absorption,Magic Resistance,Dimensional Awareness,Intangibility,Element Control,Wind Control,Echolocation,Radar Sense,Resurrection,The Force,Possession,Astral Travel,Summoning,Banish,Adaptation,Electrical Transport,Speed Force,Omnipotent,Grim Reaping,Reality Warping,Portal Creation,Vitakinesis,Projection,Insanity,Plant Control,Photographic Reflexes,Anti-Gravity,Power Nullifier,Hair Manipulation,Weather Control,Nova Force,Odin Force,Phoenix Force,Power Sense,Qwardian Power Ring,Melting,Changing Armor,Terrakinesis
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Male,Human,good,No 

# Part 2

**II. Use your combined DataFrame to answer the following questions.**

1. Compare the average weight of super powers who have Super Speed to those who do not.
2. What is the average height of heroes for each publisher?


## 1. Compare the average weight of super powers who have Super Speed to those who do not.

In [44]:
hero.groupby('Super Speed')['Weight'].mean()

Super Speed
0    126.364574
1    129.404040
Name: Weight, dtype: float64

## 2. What is the average height of heroes for each publisher?

In [45]:
hero.groupby('Publisher')['Height'].mean()

Publisher
DC Comics            187.582386
Dark Horse Comics    181.486667
George Lucas         154.692308
Image Comics         211.000000
Marvel Comics        201.015327
Shueisha             171.012658
Star Trek            182.500000
Team Epic TV         181.692308
Unknown              178.000000
Name: Height, dtype: float64